In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('movie/movies_metadata.csv')

C:\Users\B-15\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
adult                    45466 non-null object
belongs_to_collection    4494 non-null object
budget                   45466 non-null object
genres                   45466 non-null object
homepage                 7782 non-null object
id                       45466 non-null object
imdb_id                  45449 non-null object
original_language        45455 non-null object
original_title           45466 non-null object
overview                 44512 non-null object
popularity               45461 non-null object
poster_path              45080 non-null object
production_companies     45463 non-null object
production_countries     45463 non-null object
release_date             45379 non-null object
revenue                  45460 non-null float64
runtime                  45203 non-null float64
spoken_languages         45460 non-null object
status                   45379 non-null objec

In [4]:
data=data[data['overview'].isnull()==False].reset_index(drop=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44512 entries, 0 to 44511
Data columns (total 24 columns):
adult                    44512 non-null object
belongs_to_collection    4435 non-null object
budget                   44512 non-null object
genres                   44512 non-null object
homepage                 7765 non-null object
id                       44512 non-null object
imdb_id                  44497 non-null object
original_language        44502 non-null object
original_title           44512 non-null object
overview                 44512 non-null object
popularity               44507 non-null object
poster_path              44166 non-null object
production_companies     44509 non-null object
production_countries     44509 non-null object
release_date             44438 non-null object
revenue                  44506 non-null float64
runtime                  44506 non-null float64
spoken_languages         44506 non-null object
status                   44441 non-null objec

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

# TfidfVectorizer 객체 생성
tfidf=TfidfVectorizer(stop_words='english')

tfidf_matrix=tfidf.fit_transform(data['overview'])

# 영화간 유사도 구하기
cos_sim=linear_kernel(tfidf_matrix, tfidf_matrix) # 모든 영화에 대한 유사도

# 영화 title을 인덱스로 하는 Series 생성
res=pd.Series(data.index, index=data['title'])

In [7]:
def get_recommender(title, cos_sim=cos_sim) :
    idx=res[title]
    sim_scores=list(enumerate(cos_sim[idx]))
    sim_scores=sorted(sim_scores, key=lambda x:x[1], reverse=True)
    sim_scores=sim_scores[1:26]
    movie_res=[i[0] for i in sim_scores]
    return data["title"].iloc[movie_res]
    
print(get_recommender('Toy Story').index)
cos_idx=get_recommender('Toy Story').index

Int64Index([15282,  2979, 10271, 24316, 23646, 28893, 42572, 37778, 41893,
             8303, 26945,  1058, 35513, 39519,  1916, 26048, 29052, 11367,
            34082,  3039, 21201,   483, 23336, 31643, 17102],
           dtype='int64')


In [8]:
overview_25=data.loc[cos_idx]['overview'].values
overview_25

array(["Woody, Buzz, and the rest of Andy's toys haven't been played with in years. With Andy about to go to college, the gang find themselves accidentally left at a nefarious day care center. The toys must band together to escape and return home to Andy.",
       "Andy heads off to Cowboy Camp, leaving his toys to their own devices. Things shift into high gear when an obsessive toy collector named Al McWhiggen, owner of Al's Toy Barn kidnaps Woody. Andy's toys mount a daring rescue mission, Buzz Lightyear meets his match and Woody has to decide where he and his heart truly belong.",
       "Andy Stitzer has a pleasant life with a nice apartment and a job stamping invoices at an electronics store. But at age 40, there's one thing Andy hasn't done, and it's really bothering his sex-obsessed male co-workers: Andy is still a virgin. Determined to help Andy get laid, the guys make it their mission to de-virginize him. But it all seems hopeless until Andy meets small business owner Trish, a

In [9]:
string=""
for view in overview_25 :
    if string=="" :
        string+=view
    else :
        string+=" "+view
print(string)

Woody, Buzz, and the rest of Andy's toys haven't been played with in years. With Andy about to go to college, the gang find themselves accidentally left at a nefarious day care center. The toys must band together to escape and return home to Andy. Andy heads off to Cowboy Camp, leaving his toys to their own devices. Things shift into high gear when an obsessive toy collector named Al McWhiggen, owner of Al's Toy Barn kidnaps Woody. Andy's toys mount a daring rescue mission, Buzz Lightyear meets his match and Woody has to decide where he and his heart truly belong. Andy Stitzer has a pleasant life with a nice apartment and a job stamping invoices at an electronics store. But at age 40, there's one thing Andy hasn't done, and it's really bothering his sex-obsessed male co-workers: Andy is still a virgin. Determined to help Andy get laid, the guys make it their mission to de-virginize him. But it all seems hopeless until Andy meets small business owner Trish, a single mom. A fast food res

In [10]:
f=open('movie/overviews.txt', 'w', encoding='utf-8')
f.write(string)
f.close()

In [11]:
import codecs
import os
import collections
from six.moves import cPickle
import numpy as np


class TextLoader():
    def __init__(self, data_dir, batch_size, seq_length, encoding='utf-8'):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.seq_length = seq_length
        self.encoding = encoding

        input_file = os.path.join(data_dir, "overviews.txt")
        vocab_file = os.path.join(data_dir, "vocab.pkl")
        tensor_file = os.path.join(data_dir, "data.npy")

        if not (os.path.exists(vocab_file) and os.path.exists(tensor_file)):
            print("reading text file")
            self.preprocess(input_file, vocab_file, tensor_file)
        else:
            print("loading preprocessed files")
            self.load_preprocessed(vocab_file, tensor_file)
        self.create_batches()
        self.reset_batch_pointer()

    # preprocess data for the first time.
    def preprocess(self, input_file, vocab_file, tensor_file):
#         with codecs.open(input_file, "r", encoding=self.encoding) as f:
#             sentences = f.readlines()
        with codecs.open(input_file, "r", encoding=self.encoding) as f:    
            data = f.read()
        counter = collections.Counter(data)
#         counter = collections.Counter()
        
#         data = []
#         for sentence in sentences :
#             cnt=0
#             for word in sentence.split(' ') :
#                 cnt+=1
#                 if len(sentence.split(' '))!=cnt :
#                     data.append(word)
#                     data.append(' ')
#                     counter[word]+=1
#                     counter[' ']+=1
#                 else :
#                     data.append(word)
#                     counter[word]+=1
                    
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
#         print(self.vocab)
        with open(vocab_file, 'wb') as f:
            cPickle.dump(self.chars, f)
        self.tensor = np.array(list(map(self.vocab.get, data)))
#         print(self.tensor)
#         print(sentences)
        np.save(tensor_file, self.tensor)


    # load the preprocessed the data if the data has been processed before.
    def load_preprocessed(self, vocab_file, tensor_file):
        with open(vocab_file, 'rb') as f:
            self.chars = cPickle.load(f)
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(len(self.chars))))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))
    # seperate the whole data into different batches.
    def create_batches(self):
        self.num_batches = int(self.tensor.size / (self.batch_size *
                                                   self.seq_length))

        # When the data (tensor) is too small,
        # let's give them a better error message
        if self.num_batches == 0:
            assert False, "Not enough data. Make seq_length and batch_size small."

        # reshape the original data into the length self.num_batches * self.batch_size * self.seq_length for convenience.
        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]
        xdata = self.tensor
        ydata = np.copy(self.tensor)
#         print(ydata)

        #ydata is the xdata with one position shift.
        ydata[:-1] = xdata[1:]
        ydata[-1] = xdata[0]
        self.x_batches = np.split(xdata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1),
                                  self.num_batches, 1)

    def next_batch(self):
        x, y = self.x_batches[self.pointer], self.y_batches[self.pointer]
        self.pointer += 1
        return x, y

    def reset_batch_pointer(self):
        self.pointer = 0

In [12]:
# -*- coding: utf-8 -*-
# Char-RNN 예제
# Author : solaris33
# Project URL : http://solarisailab.com/archives/2487
# GitHub Repository : https://github.com/solaris33/char-rnn-tensorflow/
# Reference : https://github.com/sherjilozair/char-rnn-tensorflow

import tensorflow as tf
import numpy as np
# from utils import TextLoader

# 학습에 필요한 설정값들을 지정합니다.
data_dir = 'movie'
#data_dir = 'data/linux'
batch_size = 10 # Training : 50, Sampling : 1
seq_length = 10 # Training : 50, Sampling : 1
hidden_size = 128   # 히든 레이어의 노드 개수 : 128
learning_rate = 0.002
num_epochs = 12
num_hidden_layers = 2 # 디폴트 : 2
grad_clip = 5   # Gradient Clipping에 사용할 임계값

# TextLoader를 이용해서 데이터를 불러옵니다.
data_loader = TextLoader(data_dir, batch_size, seq_length)
# 학습데이터에 포함된 모든 단어들을 나타내는 변수인 chars와 chars에 id를 부여해 dict 형태로 만든 vocab을 선언합니다.
chars = data_loader.chars 
vocab = data_loader.vocab
vocab_size = data_loader.vocab_size # 전체 단어개수

# 인풋데이터와 타겟데이터, 배치 사이즈를 입력받기 위한 플레이스홀더를 설정합니다.
input_data = tf.placeholder(tf.int32, shape=[None, None])  # input_data : [batch_size, seq_length])
target_data = tf.placeholder(tf.int32, shape=[None, None]) # target_data : [batch_size, seq_length])
state_batch_size = tf.placeholder(tf.int32, shape=[])      # Training : 50, Sampling : 1

# RNN의 마지막 히든레이어의 출력을 소프트맥스 출력값으로 변환해주기 위한 변수들을 선언합니다.
# hidden_size -> vocab_size
softmax_w = tf.Variable(tf.random_normal(shape=[hidden_size, vocab_size]), dtype=tf.float32)
softmax_b = tf.Variable(tf.random_normal(shape=[vocab_size]), dtype=tf.float32)

# num_hidden_layers만큼 LSTM cell(히든레이어)를 선언합니다.
cells = []
for _ in range(0, num_hidden_layers):
    cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_size)
    cells.append(cell)

# cell을 종합해서 RNN을 정의합니다.
cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

# 인풋데이터를 변환하기 위한 Embedding Matrix를 선언합니다.
# hidden_size -> vocab_size
embedding = tf.Variable(tf.random_normal(shape=[vocab_size, hidden_size]), dtype=tf.float32)
inputs = tf.nn.embedding_lookup(embedding, input_data)

# 초기 state 값을 0으로 초기화합니다.
initial_state = cell.zero_state(state_batch_size, tf.float32)

# 학습을 위한 tf.nn.dynamic_rnn을 선언합니다.
# outputs : [batch_size, seq_length, hidden_size]
outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=initial_state, dtype=tf.float32)
# ouputs을 [batch_size * seq_length, hidden_size]] 형태로 바꿉니다.
output = tf.reshape(outputs, [-1, hidden_size])

# 최종 출력값을 설정합니다.
# logits : [batch_size * seq_length, vocab_size]
logits = tf.matmul(output, softmax_w) + softmax_b
probs = tf.nn.softmax(logits)

# Cross Entropy 손실 함수를 정의합니다. 
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=target_data))

# 옵티마이저를 선언하고 옵티마이저에 Gradient Clipping을 적용합니다.
# grad_clip(=5)보다 큰 Gradient를 5로 Clippin합니다.
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.apply_gradients(zip(grads, tvars))

# 세션을 열고 학습을 진행합니다.
with tf.Session() as sess:
    # 변수들에 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())
    
    for e in range(num_epochs):
        data_loader.reset_batch_pointer()
        # 초기 상태값을 지정합니다.
        state = sess.run(initial_state, feed_dict={state_batch_size : batch_size})

        for b in range(data_loader.num_batches):
            # x, y 데이터를 불러옵니다.
            x, y = data_loader.next_batch()
#             print(y)
            # y에 one_hot 인코딩을 적용합니다. 
            y = tf.one_hot(y, vocab_size)            # y : [batch_size, seq_length, vocab_size]
            y = tf.reshape(y, [-1, vocab_size])       # y : [batch_size * seq_length, vocab_size]
            y = y.eval()

            # feed-dict에 사용할 값들과 LSTM 초기 cell state(feed_dict[c])값과 hidden layer 출력값(feed_dict[h])을 지정합니다.
            feed_dict = {input_data : x, target_data: y, state_batch_size : batch_size}
            for i, (c, h) in enumerate(initial_state):
                feed_dict[c] = state[i].c
                feed_dict[h] = state[i].h

            # 한스텝 학습을 진행합니다.
            _, loss_print, state = sess.run([train_step, loss, final_state], feed_dict=feed_dict)

            print("{}(학습한 배치개수)/{}(학습할 배치개수), 반복(epoch): {}, 손실함수(loss): {:.3f}".format(
                          e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          (e+1), 
                          loss_print))

    print("트레이닝이 끝났습니다!")   
    

    # 샘플링 시작
    print("샘플링을 시작합니다!")
    num_sampling = 40  # 생성할 글자(Character)의 개수를 지정합니다. 
    prime=u' '        # 시작 글자를 ' '(공백)으로 지정합니다.
    sampling_type = 1    # 샘플링 타입을 설정합니다.
    state = sess.run(cell.zero_state(1, tf.float32)) # RNN의 최초 state값을 0으로 초기화합니다.

    # Random Sampling을 위한 weighted_pick 함수를 정의합니다.
    def weighted_pick(weights):
        t = np.cumsum(weights)
        s = np.sum(weights)
        return(int(np.searchsorted(t, np.random.rand(1)*s)))

C:\Users\B-15\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


loading preprocessed files
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.
0(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 1, 손실함수(loss): 5.979
1(학습한 

102(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.550
103(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.386
104(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.195
105(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.333
106(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.387
107(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.465
108(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.320
109(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.293
110(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.278
111(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.555
112(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.324
113(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.532
114(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.569
115(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.457
116(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.374
117(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 2, 손실함수(loss): 3.425
118(학습한 배치개수)/708(학습할 배치

240(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.944
241(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.747
242(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.709
243(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.853
244(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.885
245(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.672
246(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.475
247(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.616
248(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.799
249(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.794
250(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.778
251(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.873
252(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.742
253(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.646
254(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.953
255(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 5, 손실함수(loss): 2.611
256(학습한 배치개수)/708(학습할 배치

375(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.526
376(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.580
377(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.455
378(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.448
379(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.560
380(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.404
381(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.478
382(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.567
383(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.535
384(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.354
385(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.290
386(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.199
387(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.431
388(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.437
389(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.510
390(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 7, 손실함수(loss): 1.423
391(학습한 배치개수)/708(학습할 배치

511(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.537
512(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.697
513(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.518
514(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.757
515(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.707
516(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.559
517(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.636
518(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.659
519(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.443
520(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.531
521(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.521
522(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.668
523(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.553
524(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.599
525(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.567
526(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 9, 손실함수(loss): 0.564
527(학습한 배치개수)/708(학습할 배치

645(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 11, 손실함수(loss): 0.116
646(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 11, 손실함수(loss): 0.279
647(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 11, 손실함수(loss): 0.180
648(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 11, 손실함수(loss): 0.148
649(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.415
650(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.191
651(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.190
652(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.136
653(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.074
654(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.070
655(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.100
656(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.064
657(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.097
658(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.154
659(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.104
660(학습한 배치개수)/708(학습할 배치개수), 반복(epoch): 12, 손실함수(loss): 0.135
661(학습한 

In [15]:
import re # 정규표현식을 사용하기 위해 import합니다.

with tf.Session() as sess:
    ret = prime      # 샘플링 결과를 리턴받을 ret 변수에 첫번째 글자를 할당합니다.
    char = prime     # Char-RNN의 첫번쨰 인풋을 지정합니다.  
    sess.run(tf.global_variables_initializer())
    for n in range(num_sampling) :
        x = np.zeros((1, 1))
        x[0, 0] = vocab[prime]

        # RNN을 한스텝 실행하고 Softmax 행렬을 리턴으로 받습니다.
        feed_dict = {input_data: x, state_batch_size : 1, initial_state: state}
        [probs_result, state] = sess.run([probs, final_state], feed_dict=feed_dict)         

        # 불필요한 차원을 제거합니다.
        # probs_result : (1,65) -> p : (65)
        p = np.squeeze(probs_result)

        # 샘플링 타입에 따라 3가지 종류로 샘플링 합니다.
        # sampling_type : 0 -> 다음 글자를 예측할때 항상 argmax를 사용
        # sampling_type : 1(defualt) -> 다음 글자를 예측할때 항상 random sampling을 사용
        # sampling_type : 2 -> 다음 글자를 예측할때 이전 글자가 ' '(공백)이면 random sampling, 그렇지 않을 경우 argmax를 사용
        if sampling_type == 0:
            sample = np.argmax(p)
        elif sampling_type == 2:
            if char == ' ':
                sample = weighted_pick(p)
            else:
                sample = np.argmax(p)
        else:
            sample = weighted_pick(p)

        pred = chars[sample]
        pred = re.sub('[,/\\-\"()\[\]!?]+','',pred) # 샘플링 결과에 포함된 특수 문자를 제거합니다.
        ret += pred+" "     # 샘플링 결과에 현재 스텝에서 예측한 글자를 추가합니다. (예를들어 pred=L일 경우, ret = HEL -> HELL)
        char = pred     # 예측한 글자를 다음 RNN의 인풋으로 사용합니다.

    print("샘플링 결과:")
    print(ret)

샘플링 결과:
 champion. South detergent slate weeks No Meanwhile daughters off hasn't classmate impossible her than features Pierre which high knows divorced. Edie upset loses CIA's Clark training grandmother. moves assumed found Sedgwick dream cheered Andy's other. sure home. leaving usual leaving 


In [ ]:
import re
re.sub('[,/\\-\'\"()\[\]]+','',"few comedian, like Redford good. seems Condorman, ordered army, young Alice. technician foe, them win inner Condorman, laid, laid, army, Milly army, Condorman, greatest man disturbed may screaming fate—orchestrated laid, begins since army, Condorman, Max Moore). women. squalid see headed ")

In [17]:
# data.index=data['imdb_id'].values

In [18]:
# data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
tt0114709,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
tt0113497,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
tt0113228,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
tt0114885,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
tt0113041,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
tt0113277,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
tt0114319,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
tt0112302,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
tt0114576,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
tt0113189,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [19]:
data.reset_index(drop=True, inplace=True)

In [20]:
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [1]:
# data[data['budget']>data['revenue']]

In [1]:
from googletrans import Translator

In [2]:
sentence=" champion. South detergent slate weeks No Meanwhile daughters off hasn't classmate impossible her than features Pierre which high knows divorced. Edie upset loses CIA's Clark training grandmother. moves assumed found Sedgwick dream cheered Andy's other. sure home. leaving usual leaving "

In [3]:
trans=  Translator()
print(trans.translate(sentence,src='en', dest='ja'))

Translated(src=en, dest=ja, text=チャンピオン。南の洗剤スレートの週いいえ一方、娘たちは離婚したことをよく知っているピエールの特徴よりも不可能な彼女を同級生にしていません。 Edieの動揺はCIAのClarkトレーニングの祖母を失います。 Sedgwickの夢がAndyの他の人を応援していることがわかったと思われる動き家に帰るいつものままに, pronunciation=None, extra_data="{'translat...")


In [4]:
trans.translate(trans.translate(sentence,src='en', dest='ja').text, dest='ko').text

'챔피언. 남쪽의 세제 슬레이트 주 아니 한편, 딸들은 이혼 한 것을 잘 알고있다 피에르의 특징보다 불가능한 그녀를 동급생하지 않습니다. Edie 동요는 CIA의 Clark 교육의 할머니를 잃게됩니다. Sedgwick의 꿈이 Andy 다른 사람을 응원하고있는 것으로 생각되는 움직임 집에 갈 평소대로'

In [5]:
from tkinter import *
from tkinter.filedialog import *
from tkinter.simpledialog import *

In [7]:
window = Tk()
messagebox.showinfo('성공', '좋아요~')
window.mainloop()